# Yerevan Winter School Tutorial
## Parameter Efficient Fine Tuning for a Low Resource Language in Colab

<div align="center">

**📚 Course Repository:** [github.com/NinaKivanani/Tutorials_low-resource-llm](https://github.com/NinaKivanani/Tutorials_low-resource-llm)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NinaKivanani/Tutorials_low-resource-llm/blob/main/3_tutorial.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-View%20Repository-blue?logo=github)](https://github.com/NinaKivanani/Tutorials_low-resource-llm)

</div>

---

**Context.**  
This hands on session shows how to adapt a compact open source LLM to a low resource language with only a tiny toy dataset.  
We will use Luxembourgish as the example low resource language, but the workflow is applicable to other languages such as Armenian.

**What you will do.**

1. Set up a Colab runtime and install libraries.
2. Load a small multilingual instruction tuned model.
3. Build a tiny English to Luxembourgish translation dataset inside the notebook.
4. Run the base model before fine tuning and inspect its behaviour.
5. Apply LoRA based parameter efficient fine tuning for a few short epochs.
6. Monitor training logs and basic metrics.
7. Compare model outputs before and after fine tuning on held out examples.
8. Reflect on what would be needed to make such fine tuning robust and reliable for real world low resource applications.

**Important note.**  
This is a demonstration.  
We work with a tiny dataset and very few training steps.  
The goal is to understand the mechanics and pitfalls, not to train a production system.


In [ ]:
# If you are in Google Colab, make sure the runtime has a GPU:
# Runtime -> Change runtime type -> Hardware accelerator -> GPU

import torch

if torch.cuda.is_available():
    print("GPU available:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected. Training will be very slow.")


In [ ]:
# Install required libraries.
# In Colab, this cell may take a couple of minutes.
!pip install -q transformers datasets accelerate peft

In [ ]:
import os
import random
import math
from dataclasses import dataclass
from typing import Dict, List

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, TaskType

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# For reproducibility
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if device == "cuda":
    torch.cuda.manual_seed_all(SEED)


## 1. Load a compact multilingual model

We choose a relatively small chat tuned model so that:

- It fits in Colab GPU memory.
- It supports many languages, including low resource ones reasonably well.

Here we use the TinyLlama chat model (about 1.1B parameters).  
In a real project, the model choice would depend on license constraints, hardware, and language coverage.


In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# Some causal language models do not have a pad token set.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
)
model.to(device)

# Disable cache during training to avoid warnings.
model.config.use_cache = False

print("Model loaded.")

## 2. Build a tiny low resource toy dataset

We construct a minimal dataset of English to Luxembourgish translation pairs directly in the notebook.  

- We treat Luxembourgish (lb) as the low resource language.  
- In a real project, you would replace this list with real parallel data or task specific instances.  
- The tiny size is intentional so that training finishes in a few minutes for demonstration purposes.


In [ ]:
toy_data = [
    {
        "id": 1,
        "language": "lb",
        "source": "Good morning, how are you?",
        "target": "Gudde Moien, wéi geet et dir?",
    },
    {
        "id": 2,
        "language": "lb",
        "source": "Thank you very much for your help.",
        "target": "Villmools Merci fir deng Hëllef.",
    },
    {
        "id": 3,
        "language": "lb",
        "source": "I would like a coffee with milk, please.",
        "target": "Ech hätt gär eng Taass Kaffi mat Mëllech, wann ech gelift.",
    },
    {
        "id": 4,
        "language": "lb",
        "source": "Where is the train station?",
        "target": "Wou ass d'Eisebunnsstatioun?",
    },
    {
        "id": 5,
        "language": "lb",
        "source": "Today the weather is very cold.",
        "target": "Haut ass d'Wieder ganz kal.",
    },
    {
        "id": 6,
        "language": "lb",
        "source": "My name is Anna and I live in Luxembourg.",
        "target": "Ech heeschen Anna an ech wunnen zu Lëtzebuerg.",
    },
    {
        "id": 7,
        "language": "lb",
        "source": "Could you please speak a little more slowly?",
        "target": "Kanns du w.e.g. e bësse méi lues schwätzen?",
    },
    {
        "id": 8,
        "language": "lb",
        "source": "I am learning Luxembourgish because I work here.",
        "target": "Ech léieren Lëtzebuergesch, well ech hei schaffen.",
    },
    {
        "id": 9,
        "language": "lb",
        "source": "The next bus arrives in ten minutes.",
        "target": "Den nächste Bus kënnt an zéng Minutten un.",
    },
    {
        "id": 10,
        "language": "lb",
        "source": "This food is delicious.",
        "target": "Dëst Iessen ass lecker.",
    },
    {
        "id": 11,
        "language": "lb",
        "source": "I do not understand, can you repeat that?",
        "target": "Ech verstinn net, kanns du dat widderhuelen?",
    },
    {
        "id": 12,
        "language": "lb",
        "source": "Have a nice evening.",
        "target": "Schéinen Owend nach.",
    },
]

dataset = Dataset.from_list(toy_data)
dataset

In [ ]:
# Simple split: 75 percent train, 25 percent test.
split_dataset = dataset.train_test_split(test_size=0.25, seed=SEED)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))

for example in eval_dataset:
    print(example)

## 3. Define an instruction style prompt template

We wrap each example into a simple instruction prompt so that the model sees:

- A system like description.
- The English sentence.
- A cue to produce the Luxembourgish translation.

For training, we construct a single text sequence that contains both the prompt and the target translation.  
The model learns to generate the full sequence.  
At inference time, we will provide only the prompt and ask the model to continue.


In [ ]:
PROMPT_TEMPLATE = (
    "You are a helpful assistant that translates from English to Luxembourgish.\n"
    "Translate the following sentence into Luxembourgish.\n\n"
    "English: {source}\n"
    "Luxembourgish:"
)

def format_example(example: Dict) -> Dict:
    prompt = PROMPT_TEMPLATE.format(source=example["source"])
    full_text = prompt + " " + example["target"]
    return {
        "text": full_text,
        "language": example["language"],
        "id": example["id"],
    }

formatted_train = train_dataset.map(format_example)
formatted_eval = eval_dataset.map(format_example)

for e in formatted_train.select(range(2)):
    print("----")
    print(e["text"])

## 4. Baseline model behaviour before fine tuning

Before we change any parameters, we check how the base TinyLlama model behaves on our evaluation set.

We will:

- Use only the prompt part of each example.
- Let the model generate a continuation.
- Compare the output qualitatively to the target translation.

Keep expectations realistic.  
The base model may already know some Luxembourgish, but it was not trained specifically for this task.


In [ ]:
def build_prompt(source_sentence: str) -> str:
    return PROMPT_TEMPLATE.format(source=source_sentence)

def generate_translation(model, tokenizer, source_sentence: str, max_new_tokens: int = 64) -> str:
    prompt = build_prompt(source_sentence)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated

print("### Baseline outputs before fine tuning ###\n")

for example in eval_dataset:
    src = example["source"]
    tgt = example["target"]
    generated = generate_translation(model, tokenizer, src)
    print("English:", src)
    print("Target Luxembourgish:", tgt)
    print("Model output:")
    print(generated)
    print("=" * 60)

## 5. Prepare data for causal language model training

We now convert the formatted text examples into token ids suitable for causal language modeling.

- Each training instance is a sequence of tokens.
- The model will learn to predict the next token given previous tokens.
- For simplicity, we use the same token ids as both `input_ids` and `labels`.

In a more careful setup, you might mask the loss on prompt tokens and only train on the answer part.  
Here we keep the configuration simple so that the mechanics of parameter efficient fine tuning are clear.


In [ ]:
MAX_SEQ_LENGTH = 256

def tokenize_function(example: Dict) -> Dict:
    result = tokenizer(
        example["text"],
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding="max_length",
    )
    # For simple language modeling we use the same ids as labels.
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_train = formatted_train.map(tokenize_function, remove_columns=["text", "language", "id"])
tokenized_eval = formatted_eval.map(tokenize_function, remove_columns=["text", "language", "id"])

print(tokenized_train[0])

In [ ]:
# Data collator for causal language modeling. No masked language modeling.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

## 6. Configure LoRA parameter efficient fine tuning

Instead of updating all model parameters, we use LoRA:

- LoRA adds small trainable matrices (low rank adapters) to selected linear layers.
- The base model weights stay frozen.
- This makes fine tuning lighter and more feasible on modest hardware.
- It also reduces the risk of catastrophic forgetting.

We choose a small rank and apply LoRA to attention projection layers only.  
This is a typical starting point for LLaMA like models.


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"],  # typical for LLaMA family models
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

## 7. Training configuration

We set very conservative training hyper parameters:

- Small batch size.
- A few epochs over a tiny dataset.
- No checkpoint saving to keep the run light.
- Logging every step so that you can watch the loss.

In a realistic low resource project you would:

- Use many more examples.
- Run for longer.
- Tune hyper parameters carefully.
- Monitor validation loss and task specific metrics.


In [ ]:
output_dir = "tiny_llama_lb_lora"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    logging_steps=1,
    evaluation_strategy="epoch",
    save_strategy="no",
    weight_decay=0.0,
    fp16=(device == "cuda"),
    report_to="none",
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Trainer created.")

In [ ]:
train_result = trainer.train()
print("\nTraining completed.")
print(train_result)

eval_metrics = trainer.evaluate()
print("\nEvaluation metrics:")
print(eval_metrics)

## 8. Compare outputs before and after fine tuning

Now we generate translations again using the fine tuned model.  
We keep the prompts identical and inspect:

- Whether the model is more likely to produce Luxembourgish.
- Whether the translations are closer to our target references.
- Any side effects such as overfitting to the tiny dataset style.


In [ ]:
print("### Outputs after LoRA fine tuning ###\n")

for example in eval_dataset:
    src = example["source"]
    tgt = example["target"]
    prompt = build_prompt(src)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = peft_model.generate(
            **inputs,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("English:", src)
    print("Target Luxembourgish:", tgt)
    print("Model output after fine tuning:")
    print(generated)
    print("=" * 60)

## 9. Quick discussion prompts

Discuss in small groups or write down short notes.

1. **Data size and quality.**  
   - We used 12 examples.  
   - What kinds of errors or biases can appear if we deploy a system trained on such a tiny sample?  
   - How would you scale the dataset for a real project in a low resource setting?

2. **Evaluation.**  
   - We only looked at qualitative outputs and language modeling loss.  
   - Which task specific metrics would you design for a real application such as translation, classification, or dialogue for a low resource language?  
   - How would you build a reliable test set?

3. **Safety and robustness.**  
   - Fine tuning can change model behaviour in unexpected ways.  
   - What additional checks would you perform before using a fine tuned model with real users in a low resource community?

4. **Transfer to your language of interest.**  
   - Suppose you want to adapt the same pipeline to Armenian or another language.  
   - What would you need to change in this notebook?  
   - Which parts are reusable, and which parts are specific to the Luxembourgish toy dataset?

5. **Beyond LoRA.**  
   - Parameter efficient fine tuning is one piece of the puzzle.  
   - What other techniques could you combine with LoRA for low resource languages, for example prompting, retrieval augmented generation, multilingual pre training, or synthetic data generation?

Use these questions to connect the small scale exercise with the broader methodological and ethical questions of building LLMs for low resource languages.


## 10. Optional extensions (if you have time)

If you finish early, you can explore one of these directions:

1. **Add a second language.**  
   Extend the toy dataset with a few examples for another low resource language, for example Armenian or Kurdish, and see how the model behaves.

2. **Loss masking.**  
   Modify the tokenization step so that the loss is computed only on the answer part of each example and not on the prompt.

3. **Temperature sweep.**  
   Generate outputs with different sampling temperatures and reflect on how diversity and correctness trade off.

4. **Save and reload adapters.**  
   Use `peft_model.save_pretrained` to save only the LoRA adapters, then reload them on top of the base model in a fresh notebook.

These small experiments help build intuition about how parameter efficient fine tuning interacts with low resource data and multilingual models.
